<h1 style ="text-align:center; color: #004d00;">NaiveBayes Classifier from Scratch</h1>
<h2 style ="text-align:center; color: #004d00;">using Iris datsets</h2>


In [1]:
# Importing Libraries
import numpy as np
import pandas as pd

<h4 style ="text-align:center; color: #004d00;">I have used Watson studio to complete this assignment, next three cells explains fetching data from local computer to object storage and from object storage to watson studio in Jupyter Notebook, They are mandatory for Watson Studio only. You may Comment them </h4>

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
### instantiating a cloud object
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
      ibm_api_key_id=credentials_3['IBM_API_KEY_ID'],
      ibm_service_instance_id=credentials_3['IAM_SERVICE_ID'],
      ibm_auth_endpoint=credentials_3['IBM_AUTH_ENDPOINT'],
      config=Config(signature_version='oauth'),
      endpoint_url=credentials_3['ENDPOINT'])

In [4]:
### Downloading File from cloud objects
cos.download_file(Bucket=credentials_3['BUCKET'],Key='iris.data',Filename='iris.data')

<h2 style ="text-align:center; color: #004d00;">Loading and Separating Data into train and test data </h2>

In [5]:
# Reading File
df = pd.read_csv('iris.data', header = None)

In [6]:
# Lets look at file
df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [7]:
# Copying dataframe so that main dataframe is not distrurbed
train_df = df

In [8]:
# Lets check training data length
train_df.shape

(150, 5)

In [9]:
# Randomly seleectig a sample of 10 from data for testing and rest as training data
test_df = train_df.sample(10, replace = False)

In [10]:
# Looking at test data
test_df.head(10)

,0,1,2,3,4
12,4.8,3.0,1.4,0.1,Iris-setosa
141,6.9,3.1,5.1,2.3,Iris-virginica
132,6.4,2.8,5.6,2.2,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
72,6.3,2.5,4.9,1.5,Iris-versicolor
15,5.7,4.4,1.5,0.4,Iris-setosa
101,5.8,2.7,5.1,1.9,Iris-virginica
38,4.4,3.0,1.3,0.2,Iris-setosa
139,6.9,3.1,5.4,2.1,Iris-virginica
126,6.2,2.8,4.8,1.8,Iris-virginica


In [11]:
# Removing the test data from training data
train_df = train_df.drop(test_df.index)

In [12]:
# View train data length again ti make sure test data is removed from train data
train_df.shape

(140, 5)

In [13]:
# Resetting train data index
train_df = train_df.reset_index(drop=True)

In [17]:
train_df.head(10)

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
7,5.0,3.4,1.5,0.2,Iris-setosa
8,4.4,2.9,1.4,0.2,Iris-setosa
9,4.9,3.1,1.5,0.1,Iris-setosa


In [14]:
# Resetting test data index
test_df = test_df.reset_index(drop=True)

In [15]:
# Lets Look at test data again to view index
test_df.head(10)

,0,1,2,3,4
0,4.8,3.0,1.4,0.1,Iris-setosa
1,6.9,3.1,5.1,2.3,Iris-virginica
2,6.4,2.8,5.6,2.2,Iris-virginica
3,6.3,2.5,5.0,1.9,Iris-virginica
4,6.3,2.5,4.9,1.5,Iris-versicolor
5,5.7,4.4,1.5,0.4,Iris-setosa
6,5.8,2.7,5.1,1.9,Iris-virginica
7,4.4,3.0,1.3,0.2,Iris-setosa
8,6.9,3.1,5.4,2.1,Iris-virginica
9,6.2,2.8,4.8,1.8,Iris-virginica


In [19]:
# splitting train data into predictor and target variables.
train_X = train_df.iloc[: , [0, 1, 2, 3]].values

In [20]:
# Lets view tain_X predictor data
train_X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5. , 3.2, 1.2, 0.2],
       [5.5, 3

In [21]:
# Now separating target variable in train data
train_y = train_df.iloc[:, 4].values

In [22]:
train_y

array(['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa',
       'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',
       'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor',


In [23]:
# smilarly we can separate our test data
test_X = test_df.iloc[: , [0, 1, 2, 3]].values

In [24]:
# Lets view Test data
test_X

array([[4.8, 3. , 1.4, 0.1],
       [6.9, 3.1, 5.1, 2.3],
       [6.4, 2.8, 5.6, 2.2],
       [6.3, 2.5, 5. , 1.9],
       [6.3, 2.5, 4.9, 1.5],
       [5.7, 4.4, 1.5, 0.4],
       [5.8, 2.7, 5.1, 1.9],
       [4.4, 3. , 1.3, 0.2],
       [6.9, 3.1, 5.4, 2.1],
       [6.2, 2.8, 4.8, 1.8]])

In [25]:
# We need test data class variable to calculate accuracy 
test_y = test_df.iloc[:, 4].values

In [26]:
# Lets view test target variable
test_y

array(['Iris-setosa', 'Iris-virginica', 'Iris-virginica',
       'Iris-virginica', 'Iris-versicolor', 'Iris-setosa',
       'Iris-virginica', 'Iris-setosa', 'Iris-virginica',
       'Iris-virginica'], dtype=object)

<h2 style ="text-align:center; color: #004d00;">Creating the required Distribitions </h2>

In [27]:
# Creating NaiveBayes Class
class NaiveBayes:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self._classes = np.unique(y)
        n_classes = len(self._classes)
        #calculating mean, standard deviation, and prior propability
        self._mean = np.zeros((n_classes, n_features), dtype = np.float64)
        self._var = np.zeros((n_classes, n_features), dtype = np.float64)
        self._priors = np.zeros(n_classes, dtype = np.float64)
        
        for idx, c in enumerate(self._classes):
            X_c = X[y==c]
            self._mean[idx, :] = X_c.mean(axis = 0)
            self._var[idx, :] = X_c.var(axis = 0)
            self._priors[idx] = X_c.shape[0] / float(n_samples)
            
    def predict(self, X):
        y_pred = [self.prob_Conditional_f(x) for x in X]
        return np.array(y_pred)
                  
    def prob_Conditional_f(self, x):
        posteriors = []
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            conditional = np.sum(np.log(self.prob_Density_f(idx, x)))
            posterior = (prior + conditional)
            posteriors.append(posterior)
        return self._classes[np.argmax(posteriors)]
    def prob_Density_f(self, class_idx, x):
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numer = np.exp(- (x-mean)**2 / (2 * var))
        denom = np.sqrt(2 * np.pi * var)
        return numer / denom

In [29]:
# now lets train over model to iris train data
nb_model = NaiveBayes()
nb_model.fit(train_X, train_y)

<h2 style ="text-align:center; color: #004d00;">Prediction on Test Data and Model Accuracy </h2>

In [30]:
# making prediction on our test data
prediction = nb_model.predict(test_X)

In [ ]:
# accuracy method to check prediction accuracy.
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy*100

In [33]:
# lets calculate accuracy of our model on testing data
ac = accuracy(test_y, prediction)
print("Naive Bayes model on Iris data accuracy is ", ac,"%")

Naive Bayes model on Iris data accuracy is  100.0 %
